In [56]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
index = pd.read_csv("lego/index.csv")
meta = pd.read_csv("lego/metadata.csv", encoding="latin-1")

In [29]:
df = pd.merge(index, meta[['class_id', 'minifigure_name']], on='class_id')

In [35]:
train_df, valid_df = train_test_split(df, test_size=0.3)

In [37]:
train_df.shape, valid_df.shape

((259, 3), (112, 3))

In [38]:
train_data = []
train_data = np.array(train_df["class_id"]) - 1 
train_paths = train_df["path"].values

for i in range(259):
    img = cv2.imread("lego/" + train_paths[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    img = img / 255.0
    train_data.append(img)

In [40]:
valid_data = []
valid_label = np.array(valid_df["class_id"]) - 1 
valid_paths = valid_df["path"].values

for i in range(112):
    img = cv2.imread("lego/" + valid_paths[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    img = img / 255.0
    valid_data.append(img)

In [42]:
train_data = np.array(train_data)
valid_data = np.array(valid_data)

In [43]:
train_data.shape, train_label.shape

((259, 512, 512, 3), (259,))

In [44]:
valid_data.shape, valid_label.shape

((112, 512, 512, 3), (112,))

In [ ]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_data, 
    train_label, 
    epochs=50, 
    validation_data=(valid_data, valid_label), 
    shuffle=True, 
    batch_size=4, 
)

In [72]:
history_df = pd.DataFrame(history.history)
history_df.head()

,loss,accuracy,val_loss,val_accuracy
0,0.0,0.026954,0.0,0.025316
1,0.0,0.026954,0.0,0.025316
2,0.0,0.026954,0.0,0.025316
3,0.0,0.026954,0.0,0.025316
4,0.0,0.026954,0.0,0.025316


In [ ]:
model.save("lego.h5")

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["train", "val"])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["train", "val"])
plt.show()